# ADA RQ3

In [2]:
from google.colab import drive
drive.mount('/content/drive')
#Change directoty to MyDrive
import os
os.chdir("/content/drive/MyDrive")
#! unzip listings.csv.zip
#! unzip reviews.csv.zip

ModuleNotFoundError: No module named 'google.colab'

## Import libraries

In [1]:
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

import pandas as pd
import nltk
!pip install textblob
from textblob import TextBlob
from nltk.corpus import stopwords
!pip install spacy
from spacy.lang.en import English
import spacy
!pip install swifter
import swifter
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yashn\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Import Data and initialize variables

In [2]:
!python -m spacy download en_core_web_sm
!python -m spacy validate
# Load the datasets
listings = pd.read_csv('Tlistings.csv')
reviews = pd.read_csv('Treviews.csv')

# Data pre-processing
nltk.download('stopwords')
stop_words = stopwords.words('english')
nlp = spacy.load('en_core_web_sm')
parser = English()
reviews.dropna(subset=['comments'], inplace=True)
#choose the data size, the remaining variable name may need changes
#reviews_2022_2023 = reviews[reviews['date'].str[:4].isin(['2022', '2023'])]
reviews_2023 = reviews[reviews['date'].str[:4].isin(['2023'])]

     --------------------------------------- 12.8/12.8 MB 36.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')

✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.5.1) =================
ℹ spaCy installation:
c:\Users\yashn\anaconda3\lib\site-packages\spacy

NAME             SPACY            VERSION                            
en_core_web_sm   >=3.5.0,<3.6.0   3.5.0   ✔



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yashn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Test the listings data

In [3]:
# print the first 5 rows of the reviews dataset
#reviews.head()
# print the host_is_superhost column of the listings dataset
listings['host_is_superhost']


0        f
1        f
2        f
3        t
4        f
        ..
17992    f
17993    f
17994    f
17995    f
17996    t
Name: host_is_superhost, Length: 17997, dtype: object

## Print the shape of the listings data to see how many rows and columns there are

In [4]:
print(reviews.shape)

(431787, 6)


## 

## Text Preprocessing: Tokenizing and Cleaning Text Data using spaCy in Reviews DataFrame

In [5]:
def tokenize_text(text):
    text = text.replace('<br/>', ' ')
    doc = nlp(text)
    return [token.lemma_.lower().strip() for token in doc if not token.is_stop and not token.is_punct]

reviews.loc[:, 'clean_text'] = reviews['comments'].swifter.progress_bar(True).apply(tokenize_text)


Pandas Apply:   0%|          | 0/431787 [00:00<?, ?it/s]

In [6]:
# check head of the reviews dataset
reviews.head()


,listing_id,id,date,reviewer_id,reviewer_name,comments,clean_text
0,1419,38924112,2015-07-19,11308465,Marcela,Having the opportunity of arriving to Alexandr...,"[have, opportunity, arrive, alexandra, house, ..."
1,1419,44791978,2015-08-29,9580285,Marco,We have no enough words to describe how beauty...,"[word, describe, beautyful, cozy, alexandra, h..."
2,1419,45957133,2015-09-07,38394721,Andrea,The listing was exceptional and an even better...,"[listing, exceptional, well, experience, perso..."
3,1419,67295154,2016-03-28,3515044,Shaun,Alexandra's home was amazing and in such a nea...,"[alexandra, home, amazing, neat, neighbourhood..."
4,1419,177702208,2017-08-03,13987100,Kate,Beautiful home. Very comfortable and clean. Pe...,"[beautiful, home, comfortable, clean, perfect,..."


## Aspect-Based Sentiment Analysis on Reviews Using TextBlob

In [78]:
# Aspect extraction
aspects = ['clean', 'location', 'value', 'communication', 'accurate']

def extract_aspect_sentiment(text, aspect):
    aspect_sentiment = []
    text = ' '.join(text) # convert list to string
    
    for sentence in TextBlob(text).sentences:
        if aspect in sentence:
            sentiment = sentence.sentiment.polarity
            aspect_sentiment.append(sentiment)
    return aspect_sentiment


for aspect in aspects:
    reviews[f'{aspect}_sentiment'] = reviews['clean_text'].apply(lambda x: extract_aspect_sentiment(x, aspect))


In [84]:
# Print the number of reviews for each aspect
print(listings['host_is_superhost'].value_counts())


f    13483
t     4513
Name: host_is_superhost, dtype: int64


In [80]:
print("Unique listing_ids in reviews:", reviews['listing_id'].nunique())
print("Unique ids in listings:", listings['id'].nunique())


Unique listing_ids in reviews: 13002
Unique ids in listings: 17997


In [63]:
reviews

,listing_id,id,date,reviewer_id,reviewer_name,comments,clean_text,cleanliness_sentiment,location_sentiment,value_sentiment,communication_sentiment
0,1419,38924112,2015-07-19,11308465,Marcela,Having the opportunity of arriving to Alexandr...,"[have, opportunity, arrive, alexandra, house, ...",[],[],[],[]
1,1419,44791978,2015-08-29,9580285,Marco,We have no enough words to describe how beauty...,"[word, describe, beautyful, cozy, alexandra, h...",[],[],[],[]
2,1419,45957133,2015-09-07,38394721,Andrea,The listing was exceptional and an even better...,"[listing, exceptional, well, experience, perso...",[],[],[],[]
3,1419,67295154,2016-03-28,3515044,Shaun,Alexandra's home was amazing and in such a nea...,"[alexandra, home, amazing, neat, neighbourhood...",[],[],[],[]
4,1419,177702208,2017-08-03,13987100,Kate,Beautiful home. Very comfortable and clean. Pe...,"[beautiful, home, comfortable, clean, perfect,...",[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...
431835,836257685033296212,839846359032272671,2023-03-04,451304649,Windy,"Great location, great condo, great host","[great, location, great, condo, great, host]",[],[0.8000000000000002],[],[]
431836,836294262707555222,837689490696375542,2023-03-01,76227218,Shanice,I had an amazing stay at Dziyana’s. It was so ...,"[amazing, stay, dziyana, clean, spacious, cent...",[],[],[],[0.22111111111111115]
431837,836294262707555222,838414480217302205,2023-03-02,64795429,Sera,This apartment is located at a prime location ...,"[apartment, locate, prime, location, stunning,...",[],[0.22],[],[0.22]
431838,836294262707555222,839207427365377209,2023-03-03,503355138,Hicham,"Very beautiful place and nice location, guest ...","[beautiful, place, nice, location, guest, resp...",[],[0.725],[],[]


In [81]:
# get the amount of true and false values in the host_is_superhost column
print(listings['host_is_superhost'].value_counts())


f    13483
t     4513
Name: host_is_superhost, dtype: int64


## Comparison of Average Sentiments for Different Aspects among Superhosts and Non-Superhosts Based on Reviews

In [82]:
superhost_sentiments = []
non_superhost_sentiments = []

for group_name, group_df in reviews.groupby('listing_id'):
    # Get the superhost status for the current group, they are true or false values
    is_superhost = listings[listings['id'] == group_name]['host_is_superhost'].values[0]


    
    # Get the sentiment scores for the current group
    sentiment_scores = {}
    for aspect in aspects:
        aspect_sentiment = group_df[f'{aspect}_sentiment'].explode().tolist()
        sentiment_scores[aspect] = aspect_sentiment
    
    # Calculate the average sentiment score for each aspect
    aspect_avg_sentiments = {}
    for aspect, sentiment_list in sentiment_scores.items():
        if len(sentiment_list) > 0:
            aspect_avg_sentiments[aspect] = sum(sentiment_list) / len(sentiment_list)
        else:
            aspect_avg_sentiments[aspect] = None
    
    # Append the average sentiment scores to the appropriate list based on whether the host is a superhost
    if is_superhost == 't':
        superhost_sentiments.append(aspect_avg_sentiments)
    else:
        non_superhost_sentiments.append(aspect_avg_sentiments)

# Convert the lists to DataFrames
superhost_sentiments_df = pd.DataFrame(superhost_sentiments)
non_superhost_sentiments_df = pd.DataFrame(non_superhost_sentiments)

from tabulate import tabulate

# Print the mean and median sentiments for superhosts and non-superhosts as a table
print(tabulate([    ['Superhost Mean Sentiments', *superhost_sentiments_df.mean()],
    ['Superhost Median Sentiments', *superhost_sentiments_df.median()],
    ['Non-Superhost Mean Sentiments', *non_superhost_sentiments_df.mean()],
    ['Non-Superhost Median Sentiments', *non_superhost_sentiments_df.median()]
], headers=['', 'cleanliness', 'location', 'value', 'communication', 'accurate']))



                                   cleanliness    location     value    communication    accurate
-------------------------------  -------------  ----------  --------  ---------------  ----------
Superhost Mean Sentiments             0.397055    0.484549  0.400046         0.564966    0.302678
Superhost Median Sentiments           0.400134    0.477951  0.400046         0.564848    0.302678
Non-Superhost Mean Sentiments         0.397469    0.456494  0.444208         0.388753    0.356917
Non-Superhost Median Sentiments       0.406907    0.455556  0.413333         0.412054    0.355


Cleanliness: Refers to the overall cleanliness of the rental unit, including the tidiness and cleanliness of the living spaces and amenities, such as bathrooms, kitchens, and common areas.

Location: Refers to the location of the rental unit in terms of accessibility to nearby attractions, public transportation, and other amenities, as well as the overall safety and convenience of the neighborhood.

Value: Refers to the perceived value of the rental unit in relation to its price. This aspect considers factors such as the size, amenities, and quality of the rental unit and how they compare to the price being charged.

Communication: Refers to the host's communication with the guest before and during their stay. This aspect considers the clarity and responsiveness of the host's messages, as well as their helpfulness in addressing any issues or concerns that may arise during the guest's stay.

Comfort: Refers to the overall comfort of the rental unit, including the quality of the furniture, bedding, and other amenities.

Accuracy: Refers to the accuracy of the listing description and photos. This aspect considers the accuracy of the listing description and photos, as well as the accuracy of the host's responses to questions about the listing.

## Two-Sample t-Test Results for Sentiment Scores of Different Aspects between Superhosts and Non-Superhosts

In [83]:
# Perform two-sample t-test on each aspect between superhosts and non-superhosts
for aspect in aspects:
    superhost_scores = superhost_sentiments_df[aspect].dropna()
    non_superhost_scores = non_superhost_sentiments_df[aspect].dropna()
    
    # Print number of non-null values in each group for current aspect
    print(f'\n{aspect.capitalize()} non-null scores:')
    print(f'Superhosts: {len(superhost_scores)}')
    print(f'Non-superhosts: {len(non_superhost_scores)}')
    
    t_stat, p_val = ttest_ind(superhost_scores, non_superhost_scores, equal_var=False)
    
    # Print results
    print(f'\n{aspect.capitalize()} t-test results:')
    print(f'T-statistic: {t_stat:.2f}')
    print(f'p-value: {p_val:.4f}')
    
    # Interpret results
    alpha = 0.05
    
    if p_val < alpha:
        if t_stat > 0:
            print(f'There is a statistically significant difference in {aspect} sentiment between superhosts and non-superhosts (p < {alpha} and t > 0).')
        else:
            print(f'There is a statistically significant difference in {aspect} sentiment between superhosts and non-superhosts (p < {alpha} and t < 0).')
    else:
        print(f'There is no statistically significant difference in {aspect} sentiment between superhosts and non-superhosts (p >= {alpha}).')



Clean non-null scores:
Superhosts: 100
Non-superhosts: 476

Clean t-test results:
T-statistic: -0.03
p-value: 0.9775
There is no statistically significant difference in clean sentiment between superhosts and non-superhosts (p >= 0.05).

Location non-null scores:
Superhosts: 106
Non-superhosts: 595

Location t-test results:
T-statistic: 1.54
p-value: 0.1257
There is no statistically significant difference in location sentiment between superhosts and non-superhosts (p >= 0.05).

Value non-null scores:
Superhosts: 2
Non-superhosts: 26

Value t-test results:
T-statistic: -0.96
p-value: 0.3603
There is no statistically significant difference in value sentiment between superhosts and non-superhosts (p >= 0.05).

Communication non-null scores:
Superhosts: 12
Non-superhosts: 72

Communication t-test results:
T-statistic: 2.85
p-value: 0.0112
There is a statistically significant difference in communication sentiment between superhosts and non-superhosts (p < 0.05 and t > 0).

Accurate non-null

## Wilcoxon Rank-Sum Test Results for Sentiment Scores of Different Aspects between Superhosts and Non-Superhosts

In [75]:
from scipy.stats import ranksums

# Perform Wilcoxon rank-sum test on each aspect between superhosts and non-superhosts
for aspect in aspects:
    superhost_scores = superhost_sentiments_df[aspect].dropna()
    non_superhost_scores = non_superhost_sentiments_df[aspect].dropna()
    
    # Print number of non-null values in each group for current aspect
    print(f'\n{aspect.capitalize()} non-null scores:')
    print(f'Superhosts: {len(superhost_scores)}')
    print(f'Non-superhosts: {len(non_superhost_scores)}')
    
    stat, p_val = ranksums(superhost_scores, non_superhost_scores)
    
    # Print results
    print(f'\n{aspect.capitalize()} Wilcoxon rank-sum test results:')
    print(f'Z-statistic: {stat:.2f}')
    print(f'p-value: {p_val:.4f}')
    
    # Interpret results
    alpha = 0.05
    
    if p_val < alpha:
        if stat > 0:
            print(f'There is a statistically significant difference in {aspect} sentiment between superhosts and non-superhosts (p < {alpha} and z > 0).')
        else:
            print(f'There is a statistically significant difference in {aspect} sentiment between superhosts and non-superhosts (p < {alpha} and z < 0).')
    else:
        print(f'There is no statistically significant difference in {aspect} sentiment between superhosts and non-superhosts (p >= {alpha}).')



Clean non-null scores:
Superhosts: 100
Non-superhosts: 476

Clean Wilcoxon rank-sum test results:
Z-statistic: -0.50
p-value: 0.6177
There is no statistically significant difference in clean sentiment between superhosts and non-superhosts (p >= 0.05).

Location non-null scores:
Superhosts: 106
Non-superhosts: 595

Location Wilcoxon rank-sum test results:
Z-statistic: 1.40
p-value: 0.1613
There is no statistically significant difference in location sentiment between superhosts and non-superhosts (p >= 0.05).

Value non-null scores:
Superhosts: 2
Non-superhosts: 26

Value Wilcoxon rank-sum test results:
Z-statistic: -0.27
p-value: 0.7890
There is no statistically significant difference in value sentiment between superhosts and non-superhosts (p >= 0.05).

Communication non-null scores:
Superhosts: 12
Non-superhosts: 72

Communication Wilcoxon rank-sum test results:
Z-statistic: 2.70
p-value: 0.0070
There is a statistically significant difference in communication sentiment between superh